# Putting it all together
## Coupling dynamic soil and hydraulic plant architecture models
In this Jupyter Notebook, we compine plant hydraulic architecture models with dynamic soil models. 
In the first part, we couple the plant hydraulic architecture with a dynamic soil grid at the macroscale. 
In the second part, we add individual perirhizal zone models around the single root segments. 


### Coupling plant hydraulic architecture to a macroscopic soil grid
Spatial coupling of the soil and the root hydraulic architecture is done in the *Mapped-Plant class* for general soil grids. *Mapped plant* is a specialisation of the *Plant class*, which keeps track of emerging new nodes and segments and their location inside the numerical soil grid. Therefore, the *MappedPlant class* needs a function that maps each point in space to a linear index (representing a grid cell used by the numeric solver). Temporal coupling as achieved  by solving each of the modules, plant and soil, sequentially, within each time step as described in Giraud et al. (2025).  

The following example simulates root water uptake by a growing root system from a drying soil. 

First, we import all the necessary libraries and set paths to the required directories of CPlantBox and DuMux-Rosi. 


In [1]:
import timeit
from functools import partial

import matplotlib.pyplot as plt
import numpy as np

import functional.root_conductivities as rc  # hard coded conductivities
import plantbox as pb
import visualisation.vtk_plot as vp
from functional.xylem_flux import XylemFluxPython  # Python hybrid solver
from richards import RichardsWrapper  # Python part
from rosi_richards import RichardsSP  # C++ part (Dumux binding)

sourcedir = "/home/jhack/phd/CPlantBox/"

In [2]:
def soil_picker(x: float, y: float, z: float, soil_model) -> int:
    """
    Maps a 3D coordinate (x, y, z) to the corresponding soil grid cell index.

    This function wraps the Richards solver's `pick()` method, which returns
    the index of the grid cell containing the given coordinates. It is used
    to couple plant root segments to the soil model in CPlantBox simulations.

    Args:
        x (float): X-coordinate [cm].
        y (float): Y-coordinate [cm].
        z (float): Z-coordinate [cm].
        soil_model: The soil model object (e.g. a RichardsWrapper instance).

    Returns:
        int: The index of the soil grid cell containing the point (x, y, z).
    """
    return soil_model.pick([x, y, z])

def sinusoidal(t: float) -> float:
    """ 
    Sinusoidal function (used for transpiration) (integral over one day is 1).
    
    Args:
        t (float): Time [days].
    
    Returns:
        float: Sinusoidal value at time t.
    """
    return np.sin(2. * np.pi * np.array(t) - 0.5 * np.pi) + 1.

Next, we set all the necessary parameters for both the soil and the plant models, as well as numerical parameters, and initialise the models. Furthermore, we define points at which the results for the dynamic variables will be stored, in this case the soil cell in which the root collar is located. 

In [3]:
""" Parameters """
min_b = [-4., -4., -25.]
max_b = [4., 4., 0.]
cell_number = [8, 8, 25]  # [16, 16, 30]  # [32, 32, 60]
periodic = False

path = sourcedir + "modelparameter/structural/rootsystem/"
name = "Anagallis_femina_Leitner_2010" 
loam = [0.08, 0.43, 0.04, 1.6, 50]
initial = -659.8 + 12.5  # -659.8

trans = 6.4  # cm3 /day (sinusoidal)
wilting_point = -10000  # cm

sim_time = 7  # [day] 
rs_age = 10  # root system initial age
age_dependent = False  # conductivities
dt = 360. / (24 * 3600)  # [days] Time step must be very small


""" Initialize macroscopic soil model """
s = RichardsWrapper(RichardsSP())
s.initialize()
s.createGrid(min_b, max_b, cell_number, periodic)  # [cm]
s.setHomogeneousIC(initial, True)  # cm pressure head, equilibrium
s.setTopBC("noFlux")
s.setBotBC("noFlux")
s.setVGParameters([loam])
s.initializeProblem()
s.setCriticalPressure(wilting_point)

""" Initialize xylem model """
rs = pb.MappedPlant()
rs.readParameters(path + name + ".xml")
if not periodic:
    sdf = pb.SDF_PlantBox(0.99 * (max_b[0] - min_b[0]), 0.99 * (max_b[1] - min_b[1]), max_b[2] - min_b[2])
else:
    sdf = pb.SDF_PlantBox(np.Inf, np.Inf, max_b[2] - min_b[2])
rs.setGeometry(sdf)
r = XylemFluxPython(rs)
rc.init_conductivities(r, age_dependent)

""" Coupling (map indices) """
picker = partial(soil_picker, soil_model=s)
r.rs.setSoilGrid(picker)  # maps segments
r.rs.setRectangularGrid(pb.Vector3d(min_b), pb.Vector3d(max_b), pb.Vector3d(cell_number), False) # args: min, max, cellnumber, periodic
rs.initialize()
rs.simulate(rs_age, False)
r.test()  # sanity checks
nodes = r.get_nodes()
cci = picker(nodes[0, 0], nodes[0, 1], nodes[0, 2])  # collar cell index

Computed bounding box tree with 3199 nodes for 1600 grid entities in 0.0011959 seconds.
Computed bounding box tree with 3199 nodes for 1600 grid entities in 0.0017871 seconds.
MappedPlant::initializeLB 
Seed::initialize: RootSystem 

XylemFluxPython.test():
1112 nodes:
Node 0 [ 0.1  0.  -3. ]
Node 1 [ 0.  0. -3.]
Node 2 [-0.12610361  0.46355802 -3.87704723]
Node 3 [-0.03889977  0.27798491 -4.14622151]
Node 4 [-0.07125156  0.19385813 -4.19784925]
1111 segments:
Segment 0 [0 1] subType 0
Segment 1 [1 2] subType 1
Segment 2 [2 3] subType 2
Segment 3 [3 4] subType 3
Segment 4 [4 5] subType 4
Collar segment index 0
Collar segment [0 1]
0 segments with length < 1.e-5 cm
5 different root types from 0 to 4
ages from 0 to 10



In [ ]:
"""Workaround using Doussan like before; XylemFluyPython is kinda fucked"""
# Initialize root hydraulic properties and Doussan model
params = PlantHydraulicParameters()
params.read_parameters(sourcedir + "modelparameter/functional/plant_hydraulics/couvreur2012")

hm = HydraulicModel_Doussan(rs, params)
hm.wilting_point = wilting_point

peri = PerirhizalPython(hm.ms)
peri.set_soil(vg.Parameters(loam))

start_time = timeit.default_timer()
t = 0.
x_, y_ = [], []

sx = s.getSolutionHead_()          # [cm] soil matric potential
hs_ = hm.ms.getHs(sx)              # map to root segments
hsr = hs_.copy()                    # initial guess for fixed point iteration

N = round(sim_time / dt)

for i in range(N):
    # Solve xylem + root-soil interaction using fixed-point iteration
    hx = hm.solve(rs_age + t, -trans * sinusoidal(t), hsr, cells=False)
    hx_old = hx.copy()

    err = 1.e6
    c = 0
    while err > 1e-2 and c < 50:
        # interpolate soil-root interface potentials
        hsr = peri.soil_root_interface_potentials(hx[1:], sx, hm.ms.radii, peri.get_outer_radii("length")/np.array(hm.ms.radii))
        hx = hm.solve_again(rs_age + t, -trans * sinusoidal(t), hsr, cells=False)
        err = np.linalg.norm(hx - hx_old)
        hx_old = hx.copy()
        c += 1

    # Compute fluxes
    fluxes = hm.radial_fluxes(rs_age + t, hx, hsr, cells=False)
    s.setSource(hm.sumSegFluxes(fluxes))
    s.solve(dt)

    # Store outputs
    x_.append(t)
    y_.append(hm.get_transpiration(rs_age + t, hx.copy(), hsr.copy()))  # [cm3/day]

    # update soil potentials
    sx = s.getSolutionHead_()
    hsr = hm.ms.getHs(sx)

    t += dt

print("Coupled benchmark solved in", timeit.default_timer() - start_time, "s")

# VTK visualization
vp.plot_roots_and_soil(hm.ms.mappedSegments(), "pressure head", hx, s, periodic,
                        np.array(min_b), np.array(max_b), cell_number, interactiveImage=True)

# Transpiration
fig, ax1 = plt.subplots()
ax1.plot(x_, trans * sinusoidal(x_), 'k')  # potential
ax1.plot(x_, -np.array(y_), 'g')          # actual
ax2 = ax1.twinx()
ax2.plot(x_, np.cumsum(-np.array(y_) * dt), 'c--')
ax1.set_xlabel("Time [d]")
ax1.set_ylabel("Transpiration $[mL d^{-1}]$ per plant")
ax1.legend(['Potential', 'Actual', 'Cumulative'], loc='upper left')
plt.show()


In the main simulation loop, we sequentially perform (a) plant growth, (b) compute xylem pressure potential and (c) calculate the fluxes from soil grid cells into the plant roots as well as the soil matric potentials. The resulting actual transpiration at each time step is stored. Two figurs are produced, one that shows the soil matric potentials in the 3D soil domain as well as the root architecture and xylem pressure potentials, and a 2D plot that shows the actual, potential and cumulative transpiration with respect to time.  

In [22]:
""" Numerical solution """
start_time = timeit.default_timer()
x_, y_ = [], []
sx = s.getSolutionHead()  # inital condition, solverbase.py
N = round(sim_time / dt)
t = 0.

for i in range(0, N):

    rs.simulate(dt)
    collar_index = r.collar_index()  # returns the segment corresponding to the collar
    collar_sx = sx[collar_index]     # soil head at the collar node
    rx = r.solve(rs_age + t, -trans * sinusoidal(t), collar_sx, sx, True, wilting_point)


    #rx = r.solve(rs_age + t, -trans * sinusoidal(t), sx[cci], sx, True, wilting_point)  # xylem_flux.py; args: age, target transp, collar soil head, soil heads, fixed or not, wilting point
    x_.append(t)

    #collar_flux_val = r.collar_flux(rs_age + t, rx, sx)
    #y_.append(float(collar_flux_val))

    y_.append(float(r.collar_flux(rs_age + t, rx, sx)))  # exact root collar flux

    fluxes = r.soilFluxes(rs_age + t, rx, sx, False)
    s.setSource(fluxes)  # richards.py
    s.solve(dt)
    sx = s.getSolutionHead()  # richards.py

    min_sx, min_rx, max_sx, max_rx = np.min(sx), np.min(rx), np.max(sx), np.max(rx)
    n = round(float(i) / float(N) * 100.)
    print(
        f"[{'*' * n}{' ' * (100 - n)}] | "
        f"[{min_sx:g}, {max_sx:g}] cm soil [{min_rx:g}, {max_rx:g}] cm root at {s.simTime:g} days {rx[0]:g}"
    )
    t += dt

print (f"Coupled benchmark solved in {timeit.default_timer() - start_time} s")

""" VTK visualisation """
vp.plot_roots_and_soil(r.rs, "pressure head", rx, s, periodic, np.array(min_b), np.array(max_b), cell_number, name)

""" transpiration over time """
fig, ax1 = plt.subplots()
ax1.plot(x_, trans * sinusoidal(x_), 'k')  # potential transpiration
ax1.plot(x_, -np.array(y_), 'g')  # actual transpiration (neumann)
ax2 = ax1.twinx()
ax2.plot(x_, np.cumsum(-np.array(y_) * dt), 'c--')  # cumulative transpiration (neumann)
ax1.set_xlabel("Time [d]")
ax1.set_ylabel("Transpiration $[mL d^{-1}]$ per plant")
ax1.legend(['Potential', 'Actual', 'Cumulative'], loc = 'upper left')
np.savetxt(name, np.vstack((x_, -np.array(y_))), delimiter = ';')
plt.show()



 XylemFlux::linearSystem: conductivities failed
 organ type 3 subtype 0

IndexError: vector::_M_range_check: __n (which is 1) >= this->size() (which is 1)